In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling as RioResampling

def ensure_epsg3857(input_path, output_path=None):
    """
    Check if a GeoTIFF is in EPSG:3857. If not, reproject it to EPSG:3857.
    Returns the path to the (possibly reprojected) file.
    If output_path is None and reprojection is needed, appends '_3857.tif' to input_path.
    """

    dst_crs = "EPSG:3857"
    with rasterio.open(input_path) as src:
        if src.crs and src.crs.to_epsg() == 3857:
            return input_path  # Already in EPSG:3857
        if output_path is None:
            output_path = input_path.replace('.tif', '_3857.tif')
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=RioResampling.bilinear
                )
    return output_path